# Selective Breeding in Rice with ChromaX (JAX)

This Colab-ready notebook walks through a minimal selective breeding pipeline for rice using **ChromaX**, a JAX-based breeding simulator. Each step is explained in text and backed by code cells with inline `#` comments.

## 1) Install dependencies

We install ChromaX and the JAX stack that ChromaX builds on. In Colab, this is typically all you need.

In [ ]:
# Install ChromaX and JAX (Colab-friendly)
!pip -q install chromax jax jaxlib

## 2) Import libraries and set global configuration

We import ChromaX and JAX, and define a random seed so the breeding simulation is reproducible.

In [ ]:
# Core scientific stack
import jax
import jax.numpy as jnp

# ChromaX API (adapt imports if your installed version differs)
from chromax import Genome, TraitArchitecture, Population, BreedingProgram

# Reproducible randomness
seed = 42
key = jax.random.PRNGKey(seed)

## 3) Define a rice genome and trait architecture

We create a simplified rice genome and specify the trait architecture (e.g., yield) with additive effects.

In [ ]:
# Define a simplified rice genome: 12 chromosomes, 1,000 markers each
n_chromosomes = 12
markers_per_chr = 1000

# Create the genome object for ChromaX
rice_genome = Genome(
    n_chromosomes=n_chromosomes,
    markers_per_chromosome=markers_per_chr,
    recombination_rate=1e-8,  # example per-base recombination rate
)

# Define a trait architecture for yield
trait_arch = TraitArchitecture(
    n_qtl=200,              # number of quantitative trait loci
    effect_distribution="normal",  # additive effects
    heritability=0.4,       # narrow-sense heritability
)

## 4) Initialize founder population

We simulate a founder population to kick off the breeding program.

In [ ]:
# Create an initial population of 200 rice individuals
founders = Population.from_random(
    genome=rice_genome,
    n_individuals=200,
    key=key,
)

## 5) Set up a selective breeding program

We configure a breeding program that performs phenotyping, selects top individuals, and makes crosses.

In [ ]:
# Configure a breeding program with selection intensity and mating strategy
program = BreedingProgram(
    genome=rice_genome,
    trait_architecture=trait_arch,
    selection_fraction=0.2,  # top 20% selected each cycle
    mating_strategy="random_cross",
    offspring_per_cross=5,
)

## 6) Run multiple cycles of selection

We simulate several cycles, tracking mean breeding values and phenotypes.

In [ ]:
# Run 5 selection cycles and track progress
n_cycles = 5
pop = founders
cycle_stats = []

for cycle in range(1, n_cycles + 1):
    # Simulate phenotypes based on genetic values and environment
    pop = program.phenotype(pop, key=key)

    # Select top individuals by phenotype
    selected = program.select(pop)

    # Create the next generation via crossing
    pop = program.mate(selected, key=key)

    # Summarize cycle statistics
    stats = program.summarize(pop)
    stats["cycle"] = cycle
    cycle_stats.append(stats)

## 7) Summarize genetic gain

We aggregate results across cycles to show improvement in the target trait.

In [ ]:
import pandas as pd

# Convert stats into a table for inspection
stats_df = pd.DataFrame(cycle_stats)
stats_df

## 8) Plot mean phenotype over cycles

This visualization shows the expected genetic gain across breeding cycles.

In [ ]:
import matplotlib.pyplot as plt

# Plot mean phenotype improvement
plt.figure(figsize=(6, 4))
plt.plot(stats_df["cycle"], stats_df["mean_phenotype"], marker="o")
plt.xlabel("Cycle")
plt.ylabel("Mean Phenotype (Yield)")
plt.title("Genetic Gain Across Cycles")
plt.grid(True)
plt.show()